In [169]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
import numpy as np
import pickle 
import argparse
import cv2
import os
import random
from skimage.feature import hog
from sklearn import svm
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
sift = cv2.SIFT_create(nfeatures=100,contrastThreshold=0.01,nOctaveLayers=5)

 
# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [170]:
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [171]:
def show_images(images,titles=None):
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [172]:
def HOG(img):
    
    resized_img_1 = cv2.resize(img, (128*4, 64*4))

    # creating hog features
    ft = hog(resized_img_1, orientations=18, pixels_per_cell=(
        8, 8), cells_per_block=(2, 2), visualize=False)
    # console.log(ft)
    ft = ft.flatten()
    return ft.flatten()

In [173]:
dico = []
def load_dataset():
    path_to_dataset = r'binary'
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)

        kp, des = sift.detectAndCompute(img, None)
        if(des is not None):
            for d in des:
                dico.append(d)
            
        # show an update every 1,000 images
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
        # features.append(HOG(gray_img))
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

In [174]:
 # Load dataset with extracted features
print('Loading dataset. This will take time ...')
hog_features, labels = load_dataset()
print('Finished loading dataset.')



Loading dataset. This will take time ...


[INFO] processed 1000/7509
[INFO] processed 2000/7509
[INFO] processed 3000/7509
[INFO] processed 4000/7509
[INFO] processed 5000/7509
[INFO] processed 6000/7509
[INFO] processed 7000/7509
Finished loading dataset.


In [175]:

path_to_dataset = r'binary'

img_filenames = os.listdir(path_to_dataset)

k = 1600

batch_size = np.size(os.listdir(path_to_dataset)) * 3
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(dico)
with open( 'kmeans.pkl', 'wb') as file:
          pickle.dump(kmeans, file)

c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


Init 1/3 with method k-means++
Inertia for init 1/3: 2562936310.0
Init 2/3 with method k-means++
Inertia for init 2/3: 2577697908.0
Init 3/3 with method k-means++
Inertia for init 3/3: 2577468228.0
[MiniBatchKMeans] Reassigning 7 cluster centers.
Minibatch step 1/275: mean batch inertia: 41392.276823367516
[MiniBatchKMeans] Reassigning 3 cluster centers.
Minibatch step 2/275: mean batch inertia: 33002.573100598165, ewa inertia: 33002.573100598165
Minibatch step 3/275: mean batch inertia: 31845.782621337163, ewa inertia: 32162.040922218246
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch step 4/275: mean batch inertia: 31245.582950026735, ewa inertia: 31496.136050445893
[MiniBatchKMeans] Reassigning 1 cluster centers.
Minibatch step 5/275: mean batch inertia: 31120.028843968576, ewa inertia: 31222.85388290357
Minibatch step 6/275: mean batch inertia: 31132.476376653394, ewa inertia: 31157.184944952478
Minibatch step 7/275: mean batch inertia: 30838.961746520366, ewa inertia: 3

In [ ]:

histo_list = []
kmeans.verbose = False

path_to_dataset = r'binary'
labels = []
img_filenames = os.listdir(path_to_dataset)

for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue
        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        kp, des = sift.detectAndCompute(img, None)
        label = fn.split('.')[0]
        labels.append(label)
        histo = np.zeros(k)

        nkp = np.size(kp)
        if(des is not None):
            for d in des:
                idx = kmeans.predict([d])
                histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

        histo_list.append(histo)

In [ ]:

sift_classifier = svm.SVC(C=1.0, gamma='scale')
sift_classifier.fit(histo_list, labels)
with open( 'sift_classifier.pkl', 'wb') as file:
          pickle.dump(sift_classifier, file)

In [ ]:
KNN= KNeighborsClassifier(n_neighbors=1)
KNN.fit(histo_list, labels)
with open( 'KNN.pkl', 'wb') as file:
          pickle.dump(KNN, file)

In [ ]:
# This function will test all our classifiers on a specific feature set
# hog_classifier = svm.SVC(C=1.0, gamma='scale')

# hog_classifier.fit(hog_features, labels)
# with open( 'hog_classifier.pkl', 'wb') as file:
#           pickle.dump(hog_classifier, file)
